Import package and Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
link_dat = pd.read_csv('link table 1962-2001_csv.zip',
                         parse_dates=['datadate'],
                        usecols=['datadate', 'GVKEY', 'LPERMNO', 'conm'])

In [3]:
crsp_dat = pd.read_csv('crsp returns all 1962-2001_csv.zip',
                   parse_dates=['date'],
                       usecols=['date', 'PERMNO', 'PRC', 'RET', 'SHROUT',
                                'SHRCD', 'EXCHCD', 'PRIMEXCH','DISTCD','SHRCLS'])

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
compustat_dat = pd.read_csv('compustat_fundamentals qtr 1962-2001_csv.zip', header= 0 ,
                          parse_dates=['datadate'],
                          usecols=['datadate', 'gvkey', 'exchg', 'fyearq', 
                                   'datafqtr','prccq','atq','ltq','niq','fyr'])

In [5]:
compustat = pd.read_csv('compustat_fundamentals qtr 1962-2001_csv.zip',
                          parse_dates=['datadate'])

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (10,13,17,18,20,25,26,30,33,34,35,647,648,649,650,654,656,657,664,668,669,674,675,677,679) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
crsp = pd.read_csv('crsp returns all 1962-2001_csv.zip',
                   parse_dates=['date'])

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5,6,9,18,19,38,40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
link = pd.read_csv('link table 1962-2001_csv.zip')

C:\Users\Biao Feng\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Filter data

In [7]:
# US listed companies
exchange_codes = {
    11: 'NYSE', 12: 'American', 14: 'Nasdaq', 
    15: 'Boston', 16: 'Chicago', 17: 'Pacific', 18: 'Philadelphia'}

compustat_dat = compustat_dat[compustat_dat.exchg.isin(exchange_codes)]

# Common shares --> share class = 10 or 11
crsp_dat = crsp_dat[crsp_dat.SHRCD.isin([10, 11])]

In [8]:
import datetime
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)

In [9]:
# add market cap column to crsp
crsp_dat.loc[:, 'mcap'] = np.abs(crsp_dat.SHROUT * crsp_dat.PRC)
# add the month end dates
me_date_map = dict(zip(crsp_dat.date.unique(), [last_day_of_month(pd.to_datetime(uu)) for uu in crsp_dat.date.unique()]))
crsp_dat.loc[:, 'me_date'] = [me_date_map[uu] for uu in crsp_dat.date.values]
# change column names
link_dat = link_dat.rename(columns={'GVKEY':'gvkey', 'LPERMNO':'PERMNO'})

Merge compustat_dat csv,link_dat csv and crsp_dat csv and called it full_universe

In [10]:
merged = pd.merge(compustat_dat, link_dat, on=['gvkey', 'datadate'])
merged = merged.copy()

In [11]:
merged2 = pd.merge(merged, 
                   crsp_dat[['PERMNO', 'me_date', 'mcap','DISTCD','SHRCLS']], 
                   left_on=['PERMNO', 'datadate'], 
                   right_on=['PERMNO', 'me_date'], 
                   how='left')
merged2 = merged2.copy()

In [12]:
full_universe = merged2.copy()

In [13]:
full_universe = full_universe.sort_values(['gvkey', 'datafqtr'])

Show how many duplicated rows from full_universe

In [14]:
tmp = full_universe.groupby(['gvkey', 'datafqtr']).gvkey.count()
tmp[tmp>1] # these are the cases when there are multiple data found

gvkey   datafqtr
1004    1985Q3      2
        1997Q3      2
1016    1984Q2      2
1045    1967Q2      2
1072    1978Q1      2
        1979Q3      2
1076    1992Q3      2
        1992Q4      2
        1993Q1      2
        1993Q2      2
        1993Q3      2
        1993Q4      2
        1994Q1      2
        1994Q2      2
        1994Q3      2
        1994Q4      4
        1995Q1      4
        1995Q2      2
        1995Q3      2
        1995Q4      2
        1996Q1      2
        1996Q2      2
        1996Q3      2
        1996Q4      2
        1997Q1      2
        1997Q2      2
        1997Q3      2
        1997Q4      2
        1998Q1      2
        1998Q2      2
                   ..
142248  2001Q4      2
201395  2000Q2      2
222286  2000Q4      2
        2001Q1      2
        2001Q2      2
        2001Q3      2
        2001Q4      2
223007  1996Q4      2
        1997Q1      2
        1997Q2      2
        1997Q3      2
        1997Q4      2
        1998Q1      2
        1998Q2 

build a for loop to search PERMNO value of duplicated rows

Use PERMNO and date to check different values from crsp

In [15]:
permno = [65453]
date = '1990-12-31'
for column in crsp[crsp.PERMNO.isin(permno) & (crsp.date == date)].columns:
    if len(crsp[crsp.PERMNO.isin(permno) & (crsp.date ==  date)].ix[:,column].unique()) > 1:
        print(crsp[crsp.PERMNO.isin(permno) & (crsp.date == date)].ix[:,column])

C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
C:\Users\Biao Feng\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


1726982    1990/11/29
1726983    1990/10/22
1726984    1990/10/22
Name: DCLRDT, dtype: object
1726982    5533.0
1726983    1272.0
1726984    1212.0
Name: DISTCD, dtype: float64
1726982    0.00
1726983    0.43
1726984    0.05
Name: DIVAMT, dtype: float64
1726982    0.0518
1726983    0.0000
1726984    0.0000
Name: FACPR, dtype: float64
1726982    0.0518
1726983    0.0000
1726984    0.0000
Name: FACSHR, dtype: float64


Example for fqtr column

In [16]:
full_universe[(full_universe.gvkey == 8909) & (full_universe.datafqtr == '1975Q4')]

,gvkey,datadate,fyearq,fyr,datafqtr,atq,ltq,niq,exchg,prccq,PERMNO,conm,me_date,mcap,DISTCD,SHRCLS
229262,8909,1976-03-31,1975,3,1975Q4,41.772,21.3,1.065,11.0,8.25,49517,R T E CORP,1976-03-31,42215.25,1232.0,NaN
229263,8909,1976-03-31,1975,3,1975Q4,41.772,21.3,1.065,11.0,8.25,49517,R T E CORP,1976-03-31,42215.25,1232.0,NaN


In [36]:
link_test = link[(link.GVKEY == 8909) & (link.datadate == '1976/03/31')]
link_test

,GVKEY,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,conm,curcdq,datacqtr,datafqtr,costat
320248,8909,C,01,LU,49517,25067,1969/11/10,1988/05/31,1976/03/31,1975,4,INDL,C,D,STD,R T E CORP,USD,NaN,1975Q4,I
320249,8909,C,01,LU,49517,25067,1969/11/10,1988/05/31,1976/03/31,1976,1,INDL,C,D,STD,R T E CORP,USD,1976Q1,1976Q1,I


In [22]:
compustat[(compustat.gvkey == 8909) & (compustat.datadate >= '1976/01/31')].loc[:,['GVKEY','datadate','fqtr','datafqtr']]

,GVKEY,datadate,fqtr,datafqtr
397132,NaN,1976-03-31,1,1976Q1
397133,NaN,1976-03-31,4,1975Q4
397134,NaN,1976-06-30,2,1976Q2
397135,NaN,1976-09-30,3,1976Q3
397136,NaN,1976-12-31,4,1976Q4
397137,NaN,1977-03-31,1,1977Q1
397138,NaN,1977-06-30,2,1977Q2
397139,NaN,1977-09-30,3,1977Q3
397140,NaN,1977-12-31,4,1977Q4
397141,NaN,1978-03-31,1,1978Q1


Solve issue that link table has wrong order data

In [25]:
compustat_dat1 = compustat_dat[compustat_dat.index == 397133]
compustat_dat1 = compustat_dat[compustat_dat.index == 397131].append(compustat_dat1)
compustat_dat1 = compustat_dat1.append(compustat_dat[compustat_dat.index == 397132])

In [ ]:
link_test = l

In [33]:
merged_test = pd.merge(compustat_dat1, link_test, on=['gvkey', 'datadate'])

In [34]:
merged_test[(merged_test.gvkey == 8909) & (merged_test.datadate >= '1976-01-01')]

,gvkey,datadate,fyearq,fyr,datafqtr,atq,ltq,niq,exchg,prccq,PERMNO,conm
1,8909,1976-03-31,1975,3,1975Q4,41.772,21.3,1.065,11.0,8.25,49517,R T E CORP
2,8909,1976-03-31,1975,3,1975Q4,41.772,21.3,1.065,11.0,8.25,49517,R T E CORP
3,8909,1976-03-31,1976,12,1976Q1,41.772,21.3,0.953,11.0,8.25,49517,R T E CORP
4,8909,1976-03-31,1976,12,1976Q1,41.772,21.3,0.953,11.0,8.25,49517,R T E CORP


In [ ]:
full_universe[(full_universe.gvkey == 8909) & (full_universe.datadate == '1976-03-31')]

In [ ]:
a = full_universe.drop_duplicates(full_universe.columns)
a[(a.gvkey == 8909) & (a.datadate == '1976-03-31')]

Sort columns of DISTCD

In [ ]:
full_universe_filter1 = full_universe.sort_values(['mcap'],ascending=False).sort_values(['gvkey', 'datadate','DISTCD'])

In [ ]:
full_universe_filter1=full_universe_filter1.drop_duplicates(['gvkey', 'datadate', 'fyearq', 'fyr', 'datafqtr', 'atq', 'ltq', 'niq',
       'exchg', 'prccq', 'PERMNO', 'conm', 'me_date', 'mcap','SHRCLS'])

pairs of duplicated rows left after filtering

In [ ]:
tmp_filter1 = full_universe_filter1.groupby(['gvkey', 'datafqtr']).gvkey.count()
tmp_filter1[tmp_filter1>1] # these are the cases when there are multiple data found

In [37]:
crsp.columns

Index(['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP',
       'TICKER', 'COMNAM', 'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'TRDSTAT',
       'SECSTAT', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'CUSIP', 'DCLRDT',
       'DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'PAYDT', 'RCRDDT', 'SHRFLG',
       'HSICMG', 'HSICIG', 'DISTCD', 'DIVAMT', 'FACPR', 'FACSHR', 'ACPERM',
       'ACCOMP', 'SHRENDDT', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'TRTSCD',
       'NMSIND', 'MMCNT', 'NSDINX', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET',
       'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'ALTPRC', 'SPREAD',
       'ALTPRCDT', 'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn'],
      dtype='object')

In [ ]:
'1973-03-29'